In [1]:
import pandas as pd
import csv
import os
import re

In [2]:
filter_columns = {
    'left_only': ['組別', '檔案類型', '檔案資料夾', '大小', '修改日期'],
    'right_only': ['組別_', '檔案類型_', '檔案資料夾_', '大小_', '修改日期_'],
}

In [3]:
rename_columns = {
    'left_only': '來源路徑',
    'right_only': '比較路徑',
}

In [4]:
log_df = pd.read_csv('E:/.developing_env/duplicated_files/group_by/.backup_comp/log_different.csv')
log_df.drop(['備註', 'Unnamed: 0'], axis=1, inplace=True)
log_df.set_index('結果', drop=True, inplace=True)
log_df

,檔案資料夾,來源路徑,比較路徑,次數,來源長度,比較長度
結果,,,,,,
1,artwork,D:\MAME Games\artwork\,D:\MAME Games\mame0238b_64bit\artwork\,1,16,17
2,cfg,D:\MAME Games\cfg\,D:\MAME Games\mame0238b_64bit\cfg\,1,1,3
3,chains,D:\MAME Games\artwork\bgfx\chains\,D:\MAME Games\bgfx\chains\,1,3,2
4,chains,D:\MAME Games\artwork\bgfx\chains\,D:\MAME Games\mame0238b_64bit\bgfx\chains\,3,3,2
5,crt-geom,D:\MAME Games\bgfx\shaders\dx11\chains\crt-geom\,D:\MAME Games\mame0238b_64bit\bgfx\shaders\dx1...,6,2,3
...,...,...,...,...,...,...
21600,下載書籍,D:\_下載的文件待歸類\下載書籍\,Z:\新增書籍文件\下載書籍\,1,2,5
21601,不再聽不懂 圖解網站建置與開發,D:\_eBooks 電子書 Java+JS+HTML+CSS+RWD+Node\不再聽不懂...,Z:\新增書籍文件\資訊書籍\不再聽不懂 圖解網站建置與開發\,1,3,2
21602,讓響應式 (RWD) 網頁設計變簡單 Bootstrap 開發速成 (第二版),D:\_eBooks 電子書 Java+JS+HTML+CSS+RWD+Node\讓響應式 ...,Z:\新增書籍文件\資訊書籍\讓響應式 (RWD) 網頁設計變簡單 Bootstrap 開發...,1,4,3


In [5]:
consolidated_df = pd.DataFrame()

In [6]:
for index, row in log_df.iterrows():
    if index == 28:
        print(index)
        break

28


In [7]:
left = row.to_frame().transpose().reset_index(drop=True)

In [8]:
left

,檔案資料夾,來源路徑,比較路徑,次數,來源長度,比較長度
0,hlsl,D:\MAME Games\bgfx\effects\hlsl\,D:\MAME Games\bgfx\shaders\spirv\chains\hlsl\,6,2,10


In [17]:
import numpy as np

In [22]:
compare_result = pd.read_csv(f'E:/.developing_env/duplicated_files/group_by/compare_result/{index}.csv', dtype={'組別': 'Int64', '檔案類型': str, '檔案資料夾': str, '檔案名稱': str, '大小': 'Int64', '修改日期': str, '組別_': 'Int64', '檔案類型_': str, '檔案資料夾_': str, '大小_': 'Int64', '修改日期_': str, '_merge': str})

In [23]:
compare_result

,Unnamed: 0,組別,檔案類型,檔案資料夾,檔案名稱,大小,修改日期,組別_,檔案類型_,檔案資料夾_,大小_,修改日期_,_merge
0,0,2893,.md,hlsl,README.md,207,2020/5/19 上午 11:12:00,<NA>,NaN,NaN,<NA>,NaN,left_only
1,1,2894,.json,hlsl,post.json,2521,2020/5/19 上午 11:12:00,<NA>,NaN,NaN,<NA>,NaN,left_only
2,2,<NA>,NaN,NaN,vs_chroma.bin,<NA>,NaN,2970,.bin,hlsl,1305,2020/5/19 上午 11:12:05,right_only
3,3,<NA>,NaN,NaN,vs_color.bin,<NA>,NaN,2970,.bin,hlsl,1305,2020/5/19 上午 11:12:05,right_only
4,4,<NA>,NaN,NaN,vs_defocus.bin,<NA>,NaN,2970,.bin,hlsl,1305,2020/5/19 上午 11:12:05,right_only
5,5,<NA>,NaN,NaN,vs_distortion.bin,<NA>,NaN,2970,.bin,hlsl,1305,2020/5/19 上午 11:12:05,right_only
6,6,<NA>,NaN,NaN,vs_ntsc_decode.bin,<NA>,NaN,2970,.bin,hlsl,1305,2020/5/19 上午 11:12:05,right_only
7,7,<NA>,NaN,NaN,vs_ntsc_encode.bin,<NA>,NaN,2970,.bin,hlsl,1305,2020/5/19 上午 11:12:05,right_only
8,8,<NA>,NaN,NaN,vs_phosphor.bin,<NA>,NaN,2970,.bin,hlsl,1305,2020/5/19 上午 11:12:05,right_only
9,9,<NA>,NaN,NaN,vs_post.bin,<NA>,NaN,2970,.bin,hlsl,1305,2020/5/19 上午 11:12:05,right_only


In [24]:
compare_result.rename(columns={'Unnamed: 0':'索引號'}, inplace=True)

In [33]:
compare_result.set_index('檔案名稱', inplace=True)

In [34]:
compare_result

,索引號,組別,檔案類型,檔案資料夾,大小,修改日期,組別_,檔案類型_,檔案資料夾_,大小_,修改日期_,_merge
檔案名稱,,,,,,,,,,,,
README.md,0,2893.0,.md,hlsl,207.0,2020/5/19 上午 11:12:00,NaN,NaN,NaN,NaN,NaN,left_only
post.json,1,2894.0,.json,hlsl,2521.0,2020/5/19 上午 11:12:00,NaN,NaN,NaN,NaN,NaN,left_only
vs_chroma.bin,2,NaN,NaN,NaN,NaN,NaN,2970.0,.bin,hlsl,1305.0,2020/5/19 上午 11:12:05,right_only
vs_color.bin,3,NaN,NaN,NaN,NaN,NaN,2970.0,.bin,hlsl,1305.0,2020/5/19 上午 11:12:05,right_only
vs_defocus.bin,4,NaN,NaN,NaN,NaN,NaN,2970.0,.bin,hlsl,1305.0,2020/5/19 上午 11:12:05,right_only
vs_distortion.bin,5,NaN,NaN,NaN,NaN,NaN,2970.0,.bin,hlsl,1305.0,2020/5/19 上午 11:12:05,right_only
vs_ntsc_decode.bin,6,NaN,NaN,NaN,NaN,NaN,2970.0,.bin,hlsl,1305.0,2020/5/19 上午 11:12:05,right_only
vs_ntsc_encode.bin,7,NaN,NaN,NaN,NaN,NaN,2970.0,.bin,hlsl,1305.0,2020/5/19 上午 11:12:05,right_only
vs_phosphor.bin,8,NaN,NaN,NaN,NaN,NaN,2970.0,.bin,hlsl,1305.0,2020/5/19 上午 11:12:05,right_only


In [35]:
group_compare = compare_result.groupby('_merge')

In [36]:
filter_group = iter(group_compare.groups.keys())

In [37]:
filter_name = next(filter_group)

In [38]:
filter_name

'left_only'

In [39]:
get_frame = group_compare.get_group(filter_name)[filter_columns[filter_name]]

In [40]:
get_frame

,組別,檔案類型,檔案資料夾,大小,修改日期
檔案名稱,,,,,
README.md,2893.0,.md,hlsl,207.0,2020/5/19 上午 11:12:00
post.json,2894.0,.json,hlsl,2521.0,2020/5/19 上午 11:12:00


In [41]:
get_frame.columns = [column.rstrip('_') for column in get_frame.columns]

In [42]:
get_frame

,組別,檔案類型,檔案資料夾,大小,修改日期
檔案名稱,,,,,
README.md,2893.0,.md,hlsl,207.0,2020/5/19 上午 11:12:00
post.json,2894.0,.json,hlsl,2521.0,2020/5/19 上午 11:12:00


In [43]:
get_frame['差異路徑'] = rename_columns[filter_name]

In [44]:
get_frame

,組別,檔案類型,檔案資料夾,大小,修改日期,差異路徑
檔案名稱,,,,,,
README.md,2893.0,.md,hlsl,207.0,2020/5/19 上午 11:12:00,來源路徑
post.json,2894.0,.json,hlsl,2521.0,2020/5/19 上午 11:12:00,來源路徑


In [45]:
teamed = get_frame.groupby('組別')

In [46]:
group_teamed = iter(teamed)

In [47]:
group_teamed

<generator object BaseGrouper.get_iterator at 0x000001EE96B5D548>

In [48]:
team_num, team_frame = next(group_teamed)

In [49]:
team_num

2893.0

In [50]:
team_frame

,組別,檔案類型,檔案資料夾,大小,修改日期,差異路徑
檔案名稱,,,,,,
README.md,2893.0,.md,hlsl,207.0,2020/5/19 上午 11:12:00,來源路徑


In [51]:
team_frame.reset_index(inplace=True)

In [52]:
team_frame

,檔案名稱,組別,檔案類型,檔案資料夾,大小,修改日期,差異路徑
0,README.md,2893.0,.md,hlsl,207.0,2020/5/19 上午 11:12:00,來源路徑


In [111]:
left

,檔案資料夾,來源路徑,比較路徑,次數,來源長度,比較長度
0,artwork,D:\MAME Games\artwork\,D:\MAME Games\mame0238b_64bit\artwork\,1,16,17


In [112]:
right = team_frame

In [113]:
right

,檔案名稱,組別,檔案類型,檔案資料夾,大小,修改日期,差異路徑
0,dir (2).txt,2878,.txt,artwork,26,1998/10/4 上午 02:20:58,比較路徑


In [114]:
isolated_team = left.merge(right, on='檔案資料夾')

In [115]:
isolated_team

,檔案資料夾,來源路徑,比較路徑,次數,來源長度,比較長度,檔案名稱,組別,檔案類型,大小,修改日期,差異路徑
0,artwork,D:\MAME Games\artwork\,D:\MAME Games\mame0238b_64bit\artwork\,1,16,17,dir (2).txt,2878,.txt,26,1998/10/4 上午 02:20:58,比較路徑


In [117]:
consolidated_df = pd.concat([consolidated_df, isolated_team])

In [118]:
consolidated_df

,檔案資料夾,來源路徑,比較路徑,次數,來源長度,比較長度,檔案名稱,組別,檔案類型,大小,修改日期,差異路徑
0,artwork,D:\MAME Games\artwork\,D:\MAME Games\mame0238b_64bit\artwork\,1,16,17,dir (2).txt,2878,.txt,26,1998/10/4 上午 02:20:58,比較路徑
